# Artificial Neural Network

Felix Limanta (13515065), Holy Lovenia (13515113), Agus Gunawan (13515143)

## Implementation of ANN

Artificial neural network (ANN) is made of interconnected layers of nodes, which is similar to the structures and functions of neurons in human brain.

Normally, connections between the nodes (neurons) are called as edges. These edges are associated with weights, which will adjust themselves during learning.

![Artificial neural network](https://icdn5.digitaltrends.com/image/artificial_neural_network_1-791x388.jpg)

Nodes are typically grouped into specific layers. Different layers perform different transformations on their inputs. The first layer is regarded as input layer, while the last is output layer. Output layer is used to represent the final outputs as their corresponding predicted classes. Between input and output layer, hidden layers may be present to process the inputs by applying an activation function and produce results according to the needs of output layer.

The disconnected nodes in the network are called as bias nodes, which are useful to shift the activation function to the desired direction. Below is an example of network with the presence of bias nodes.

![Network with bias](http://documentation.statsoft.com/STATISTICAHelp/SANN/Images/mlpdiagram.jpg)

In [212]:
from random import random, randint
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tnrange, trange, tgrange

import math
import numpy as np
import pandas as pd

### Layer implementation

In this step, we implement `LunakDense` class to represent the hidden layer and output layer, with parameters as listed below.

1. `units`: `int`, the number of nodes in the layer

2. `activation`: `'sigmoid'`, activation function

3. `input_dim`: `int`, dimension of the input (e.g. 2D, 3D, ...)

4. `init`: `'uniform', 'random'`, type of distribution for the initial weight matrix

5. `use_bias`: `boolean`, whether there will be bias node present or not (default=`False`)

6. `seed`: `int`, the number of random seed (default=`None`)

**Propagation function**

It computes input $p_j(t)$ to neuron $j$ from the outputs $o_i(t)$ of predecessor neurons and bias $w_{0j}$

![Sigma](https://wikimedia.org/api/rest_v1/media/math/render/svg/53a6369d6948c2a582469ed48def95b151953e9d)

In [211]:
class LunakDense:
    def __init__(self, units, activation, input_dim, init, use_bias=False, seed=None):
        self.units = units
        self.input_dim = input_dim
        
        if activation == 'sigmoid':
            self.activation_function = self.sigmoid
        else:
            print('Activation function not supported')
        
        np.random.seed(seed)
        
        if init == 'uniform':
            self.weight_matrix = np.random.uniform(-0.05, 0.05, size=(self.units, input_dim)) 
            print(self.weight_matrix)
        elif init == 'random':
            self.weight_matrix = np.random.random(size=(self.units, input_dim))
        else:
            print('Init function not supported')
        
        self.delta_weight_matrix_before = np.zeros((self.units, input_dim))
        self.delta_weight_matrix = np.zeros((self.units, input_dim))
        
        self.use_bias = use_bias
        if self.use_bias:
            bias = np.zeros((units, 1))
            self.weight_matrix = np.hstack((self.weight_matrix, bias))
            self.delta_weight_matrix_before = np.hstack((self.delta_weight_matrix_before, np.zeros((units, 1))))
            self.delta_weight_matrix = np.hstack((self.delta_weight_matrix, np.zeros((units, 1))))
            
    def calculate_sigma(self, input_list):
        if self.use_bias:
            input_list = np.append(input_list, 1)
        
        result_list = np.array([])
        for weight_neuron in self.weight_matrix:
            result_list = np.append(result_list, np.dot(weight_neuron, input_list))
        return np.array(result_list)
    
    def calculate_output(self, input_list):
        output_list = np.array([])
        for sigma_neuron in self.calculate_sigma(input_list):
            output_list = np.append(output_list, self.activation_function(sigma_neuron))
        self.output_list = output_list
        return output_list.copy()
    
    def calculate_local_gradient_output_layer(self, target_list):
        """
        Use this if the layer is output layer
        """
        result_list = np.array([])
        for index, output in enumerate(self.output_list):
            local_gradient = output * (1 - output) * (target_list[index] - output)
            result_list = np.append(result_list, local_gradient)  
        self.local_gradient = result_list
        return result_list.copy()
    
    def calculate_local_gradient_hidden_layer(self, local_gradient_output_list, output_layer_weight_matrix):
        """
        Use this if the layer is hidden layer
        """
        result_list = np.array([])
        for index, output in enumerate(self.output_list):
            sigma_local_gradient_output = 0
            for unit_number, local_gradient in enumerate(local_gradient_output_list):
                sigma_local_gradient_output += output_layer_weight_matrix[unit_number][index] * local_gradient
            error_hidden = output * (1 - output) * sigma_local_gradient_output
            result_list = np.append(result_list, error_hidden)
        self.local_gradient = result_list
        return result_list.copy()
    
    def update_delta_weight(self, lr, input_list, momentum=None):
        """
        Function to update delta weight
        """
        if self.use_bias:
            input_list = np.append(input_list, 1)
        if momentum == None:
            for j, unit in enumerate(self.weight_matrix): #j  
                for i, source in enumerate(unit): #i
                    delta_weight = lr * self.local_gradient[j] * input_list[i]
                    self.delta_weight_matrix[j][i] = delta_weight.copy()
        else:
            for j, unit in enumerate(self.weight_matrix): #j  
                for i, source in enumerate(unit): #i
                    delta_weight = lr * self.local_gradient[j] * input_list[i] + momentum * self.delta_weight_matrix_before[j][i]
                    
                    # Update Delta Weight
                    self.delta_weight_matrix_before[j][i] = delta_weight.copy()
            
            # Copy Last Update of Weight Matrix Before (Equal to Last Weight Matrix)
            for j, unit in enumerate(self.delta_weight_matrix_before):
                for i, source in enumerate(unit):
                    self.delta_weight_matrix[j][i] = self.delta_weight_matrix_before[j][i].copy()
            
    def update_weight(self):
        """
        Function to update weight
        """
        for j, unit in enumerate(self.delta_weight_matrix_before):
            for i, source in enumerate(unit):
                self.weight_matrix[j][i] += self.delta_weight_matrix[j][i]
    
    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x))

### Model implementation

ANN model is implemented using stochastic gradient descent (SGD) as the learning rule. SGD is known as a strategy for searching through a large or infinite hypothesis space.

ANN typically consists of two steps: feed-forward and backpropagation.

![ANN steps](https://www.researchgate.net/profile/Morteza_Esfandyari/publication/241741756/figure/fig2/AS:298577172680729@1448197753779/Back-propagation-multilayer-ANN-with-one-hidden-layer.png)

Feed-forward is used to predict the given inputs based on the weights the network currently has.

![Feed-forward](https://d18rbf1v22mj88.cloudfront.net/wp-content/uploads/sites/3/2018/03/13094759/neural_networks_fully_connected_layers_gumgum1.gif)

Backpropagation is used to calculate the error in each node and update the weights based on the error.

![Backpropagation](https://raw.githubusercontent.com/mtoto/mtoto.github.io/master/data/2017-11-08-net/result.gif)

The algorithm used for the feed-forward and backpropagation is based on [Machine Learning book by Tom Mitchell](https://www.cs.ubbcluj.ro/~gabis/ml/ml-books/McGrawHill%20-%20Machine%20Learning%20-Tom%20Mitchell.pdf) on page 98.

Parameters used for training (`fit` method).

1. `X`: `data`, training data

2. `y`: `data`, labels for training data

3. `epochs`: `int`, the number of epochs that will be run

4. `lr`: `float`, learning rate

5. `momentum`: `float`, momentum (used to prevent local minima)

6. `batch_size`: `int`, incremental when 1 (default=`len(X)`)

7. `val_data`: `(X_val, y_val)`, for validation purposes (default=`None`, will use `val_size`=0.1)

8. `val_size`: `float`, used to split X and y to get validation data (default=0)

In [206]:
class LunakArtificialNeuralNetwork:
    def __init__(self, loss='root_mean_squared', optimizer='sgd'):
        assert loss == 'root_mean_squared', 'loss function not supported'
        assert optimizer == 'sgd', 'optimizer not supported'
        self.layers = []
    
    def add(self, layer):
        self.layers.append(layer)
        
    def feed_forward(self, X_instance):
        # Calculate output with the first hidden layer
        output_list = self.layers[0].calculate_output(X_instance)
        # Calculate output with the second until the last layer
        for layer in self.layers[1:]:
            next_output_list = layer.calculate_output(output_list)
            output_list = next_output_list
        return output_list.copy()
            
    def backpropagation(self, y_instance):
        # Calculate local gradient for output layer
        next_local_gradient_list = self.layers[-1].calculate_local_gradient_output_layer([y_instance])
        next_layer_weight_matrix = self.layers[-1].weight_matrix

        # Calculate local gradient for hidden layer(s)
        for layer_idx, layer in enumerate(reversed(self.layers[0:-1])):
            next_local_gradient_list = layer.calculate_local_gradient_hidden_layer(next_local_gradient_list, next_layer_weight_matrix)
            next_layer_weight_matrix = layer.weight_matrix
            
    def calculate_delta_weight(self, X_instance, lr, momentum):
        # Update delta weight for first hidden layer
        self.layers[0].update_delta_weight(lr, X_instance, momentum)
        
        # Update delta weight for other layers
        for layer_idx, layer in enumerate(self.layers[1:]):
            layer.update_delta_weight(lr, self.layers[layer_idx].output_list, momentum)
    
    def fit(self, X, y, epochs, lr, momentum=None, batch_size=None, val_data=None, val_size=0):
        assert X.shape[1] == self.layers[0].input_dim, 'Input dimension must be same with the column'
        self.classes_ = np.unique(y)
        
        if batch_size == None:
            batch_size = len(X)
            
        if val_data is None:
            val_size = 0.1
            X, X_val, y, y_val = train_test_split(X, y, test_size=val_size)
        else:
            X_val = val_data[0]
            y_val = val_data[1]
        
        if val_data is not None and val_size != 0:
            print('Validation data will be used instead of val_size.')
            
        for epoch in range(epochs):
            delta = batch_size
            
            with tnrange(0, len(X), delta, desc='Epoch {}'.format(epoch + 1)) as pbar:
                for start in pbar:
                    X_batch = X[start:start+delta]
                    y_batch = y[start:start+delta]

                    for idx, X_instance in enumerate(X_batch):
                        self.feed_forward(X_instance)
                        self.backpropagation(y_batch[idx][0])
                        self.calculate_delta_weight(X_instance, lr, momentum)

                    for layer in self.layers:
                        layer.update_weight()

                    pred = self.predict(X)
                    pred_val = self.predict(X_val)

                    acc = self.calculate_accuracy(y, pred)
                    val_acc = self.calculate_accuracy(y_val, pred_val)
                    loss = mean_squared_error(y, pred)
                    val_loss = mean_squared_error(y_val, pred_val)

                    postfix = {
                        'loss': loss,
                        'val_loss': val_loss,
                        'acc': acc,
                        'val_acc': val_acc}
                    pbar.set_postfix(postfix, refresh=True)
    
    def predict_proba(self, X):
        predictions = []
        for idx, X_instance in enumerate(X):
            X_pred = self.feed_forward(X_instance)
            predictions.append([np.mean(X_pred.copy())])
        return predictions
    
    def predict(self, X):
        predictions = []
        for idx, X_instance in enumerate(X):
            X_pred_proba = self.feed_forward(X_instance)
            X_pred = min(self.classes_, key=lambda pred_class:abs(pred_class - np.mean(X_pred_proba)))
            predictions.append([X_pred.copy()])
        return predictions
    
    def calculate_accuracy(self, y_true, y_pred):
        if len(confusion_matrix(y_true, y_pred).ravel()) > 1:
            tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        else:
            tp = confusion_matrix(y_true, y_pred).ravel()[0]
            fp = 0
            fn = 0
            tn = 0
        return (tp + tn) / (tp + tn + fp + fn)

## Data Preparation

In [19]:
from scipy.io.arff import loadarff
import pandas as pd

### Read weather data

In [20]:
raw_data = loadarff('dataset/weather.arff')

In [21]:
data = pd.DataFrame(raw_data[0])

In [22]:
data.head()

,outlook,temperature,humidity,windy,play
0,b'sunny',85.0,85.0,b'FALSE',b'no'
1,b'sunny',80.0,90.0,b'TRUE',b'no'
2,b'overcast',83.0,86.0,b'FALSE',b'yes'
3,b'rainy',70.0,96.0,b'FALSE',b'yes'
4,b'rainy',68.0,80.0,b'FALSE',b'yes'


### Preprocessing

In [23]:
def convert_to_binary_vector(data):
    return pd.get_dummies(data)

In [24]:
for idx, column in enumerate(['outlook', 'windy', 'play']):
    data[column] = data[column].str.decode('utf-8')

In [25]:
bv_outlook = convert_to_binary_vector(data['outlook'])
bv_outlook.head()

,overcast,rainy,sunny
0,0,0,1
1,0,0,1
2,1,0,0
3,0,1,0
4,0,1,0


In [26]:
bv_windy = convert_to_binary_vector(data['windy'])
bv_windy.head()

,FALSE,TRUE
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0


In [27]:
preproc_data = data.drop('outlook', 1).drop('windy', 1)

In [28]:
preproc_data['play'] = preproc_data['play'].astype('category')

In [29]:
preprocessed_data = pd.concat([bv_outlook, bv_windy, preproc_data], axis=1)
preprocessed_data.head()

,overcast,rainy,sunny,FALSE,TRUE,temperature,humidity,play
0,0,0,1,1,0,85.0,85.0,no
1,0,0,1,0,1,80.0,90.0,no
2,1,0,0,1,0,83.0,86.0,yes
3,0,1,0,1,0,70.0,96.0,yes
4,0,1,0,1,0,68.0,80.0,yes


In [30]:
y = pd.DataFrame({'play': preprocessed_data['play'].cat.codes})
y.head()

,play
0,0
1,0
2,1
3,1
4,1


In [31]:
X = preprocessed_data.drop('play', 1)

In [32]:
for column in X.columns:
    X[column] = X[column].astype('float')

In [33]:
X.head()

,overcast,rainy,sunny,FALSE,TRUE,temperature,humidity
0,0.0,0.0,1.0,1.0,0.0,85.0,85.0
1,0.0,0.0,1.0,0.0,1.0,80.0,90.0
2,1.0,0.0,0.0,1.0,0.0,83.0,86.0
3,0.0,1.0,0.0,1.0,0.0,70.0,96.0
4,0.0,1.0,0.0,1.0,0.0,68.0,80.0


In [34]:
y = y.astype('float')

In [35]:
X = np.array(X)
y_new = [instance_y[0] for instance_y in np.array(y)]
y = np.array(y)

In [36]:
X

array([[ 0.,  0.,  1.,  1.,  0., 85., 85.],
       [ 0.,  0.,  1.,  0.,  1., 80., 90.],
       [ 1.,  0.,  0.,  1.,  0., 83., 86.],
       [ 0.,  1.,  0.,  1.,  0., 70., 96.],
       [ 0.,  1.,  0.,  1.,  0., 68., 80.],
       [ 0.,  1.,  0.,  0.,  1., 65., 70.],
       [ 1.,  0.,  0.,  0.,  1., 64., 65.],
       [ 0.,  0.,  1.,  1.,  0., 72., 95.],
       [ 0.,  0.,  1.,  1.,  0., 69., 70.],
       [ 0.,  1.,  0.,  1.,  0., 75., 80.],
       [ 0.,  0.,  1.,  0.,  1., 75., 70.],
       [ 1.,  0.,  0.,  0.,  1., 72., 90.],
       [ 1.,  0.,  0.,  1.,  0., 81., 75.],
       [ 0.,  1.,  0.,  0.,  1., 71., 91.]])

In [37]:
y

array([[0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.]])

## Artificial neural network experiment

### Lunak

In [207]:
lunak_ann = LunakArtificialNeuralNetwork()

In [208]:
lunak_ann.add(LunakDense(2, 'sigmoid', 7, 'uniform', False, seed=5))
lunak_ann.add(LunakDense(1, 'sigmoid', 2, 'uniform', False, seed=5))

[[-0.02780068  0.03707323 -0.02932808  0.04186109 -0.00115888  0.01117439
   0.02659079]
 [ 0.0018418  -0.02031995 -0.03122788 -0.04192587  0.02384403 -0.00586908
  -0.03416901]]
[[-0.02780068  0.03707323]]


In [209]:
lunak_ann.fit(X, y, epochs=50, momentum=0.001, lr=0.001, batch_size=2, val_size=0.2)

In [60]:
confusion_matrix([0, 0, 0], [1, 1, 1])

array([[0, 3],
       [0, 0]])

In [42]:
predictions = lunak_ann.predict(X)

In [43]:
predictions

[[1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0]]

### Keras

In [44]:
from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import RandomUniform
import keras
import pandas as pd
import tensorflow as tf

Using TensorFlow backend.


In [45]:
keras_ann = Sequential()

In [46]:
initializer = RandomUniform(minval=-0.05, maxval=0.05, seed=5)

In [47]:
keras_ann.add(Dense(2, activation='sigmoid', input_dim=7, use_bias=False, kernel_initializer=initializer))
keras_ann.add(Dense(1, activation='sigmoid', use_bias=False, kernel_initializer=initializer))

In [48]:
optimizer_ = keras.optimizers.SGD(momentum=0.001, lr=0.001)

In [49]:
keras_ann.compile(loss='mean_squared_error', optimizer=optimizer_, metrics=['accuracy'])

In [50]:
keras_ann.fit(X, y, epochs=50, batch_size=10)

Epoch 1/50
14/14 [==============================] - 1s 41ms/step - loss: 0.2495 - acc: 0.6429
Epoch 2/50
14/14 [==============================] - 0s 513us/step - loss: 0.2495 - acc: 0.6429
Epoch 3/50
14/14 [==============================] - 0s 471us/step - loss: 0.2495 - acc: 0.6429
Epoch 4/50
14/14 [==============================] - 0s 447us/step - loss: 0.2495 - acc: 0.6429
Epoch 5/50
14/14 [==============================] - 0s 342us/step - loss: 0.2495 - acc: 0.6429
Epoch 6/50
14/14 [==============================] - 0s 186us/step - loss: 0.2494 - acc: 0.6429
Epoch 7/50
14/14 [==============================] - 0s 213us/step - loss: 0.2494 - acc: 0.6429
Epoch 8/50
14/14 [==============================] - 0s 180us/step - loss: 0.2494 - acc: 0.6429
Epoch 9/50
14/14 [==============================] - 0s 263us/step - loss: 0.2494 - acc: 0.6429
Epoch 10/50
14/14 [==============================] - 0s 225us/step - loss: 0.2494 - acc: 0.6429
Epoch 11/50
14/14 [==============================]

In [51]:
keras_ann.predict(X)

array([[0.50376505],
       [0.50385934],
       [0.50383097],
       [0.5039642 ],
       [0.5037419 ],
       [0.5035906 ],
       [0.5034801 ],
       [0.5038945 ],
       [0.5034984 ],
       [0.5037456 ],
       [0.5035202 ],
       [0.5039024 ],
       [0.50364953],
       [0.50392693]], dtype=float32)